<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install wandb -qU
!wandb login 617d80dc70f383d8ff9ef1c2b94a78ccc8d8db23

     |████████████████████████████████| 1.9 MB 30.7 MB/s 
     |████████████████████████████████| 182 kB 72.0 MB/s 
     |████████████████████████████████| 168 kB 54.4 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 71.6 MB/s 
     |████████████████████████████████| 166 kB 75.6 MB/s 
     |████████████████████████████████| 166 kB 77.3 MB/s 
     |████████████████████████████████| 162 kB 69.3 MB/s 
     |████████████████████████████████| 162 kB 74.8 MB/s 
     |████████████████████████████████| 158 kB 70.3 MB/s 
     |████████████████████████████████| 157 kB 76.7 MB/s 
     |████████████████████████████████| 157 kB 74.3 MB/s 
     |████████████████████████████████| 157 kB 80.3 MB/s 
     |████████████████████████████████| 157 kB 75.8 MB/s 
     |████████████████████████████████| 157 kB 78.7 MB/s 
     |████████████████████████████████| 157 kB 75.0 MB/s 
     |████████████████████████████████| 157 kB 78.0 MB/s 
     |██████████

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
import numpy as np
from sklearn.metrics import classification_report, f1_score

In [3]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [4]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  shuffle = False,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 100 files belonging to 5 classes.


In [5]:
y = np.concatenate([y for x, y in test_ds], axis=0)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

In [6]:
labels = test_ds.class_names

In [7]:
class argmax_layer(tf.keras.layers.Layer):
  def __init__(self):
    super(argmax_layer, self).__init__()

  def call(self, inputs):
    return tf.math.argmax(inputs, axis=1)

In [64]:
#convnext
api = wandb.Api()
artifact = api.artifact('vedmanivaidya/Transfer_Learning_New/model-convnext_tiny_1k_224:v39', type='model')
artifact.download()
model = tf.keras.models.load_model("/content/artifacts/model-convnext_tiny_1k_224:v39")

wandb: Downloading large artifact model-convnext_tiny_1k_224:v39, 108.69MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:7.4


In [65]:
model.evaluate(test_ds)

4/4 [==============================] - 3s 300ms/step - loss: 0.1023 - accuracy: 0.9900


[0.10228149592876434, 0.9900000095367432]

In [66]:
seq = tf.keras.Sequential()
seq.add(model)
seq.add(argmax_layer())
preds = seq.predict(test_ds)

4/4 [==============================] - 2s 210ms/step


In [67]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(y, preds, average='macro'))

(0.9945945945945945, 0.9939393939393939, 0.9941833508956798, None)


In [30]:
'''from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
f1_score(y, preds, average='macro')'''

"from sklearn.metrics import f1_score\nfrom sklearn.metrics import precision_recall_fscore_support\nf1_score(y, preds, average='macro')"